# Initialization

In [99]:
# Imports
import numpy as np
import time
import random
from tqdm import tqdm
import os
import pandas as pd
import cv2
import tensorflow as tf
import tensorboard
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
import tensorflow as tf
from collections import deque
import PIL
from PIL import Image


In [100]:
# GPU Check
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


## Parameters Init

In [101]:
# Parameters
MAP_X = 30
MAP_Y = 20
GRID_SIZE = 0.5
SIZE_X = int(MAP_X/GRID_SIZE)
SIZE_Y = int(MAP_Y/GRID_SIZE)

pathDist_path = f'scripts/path_dist/{MAP_X}x{MAP_Y}_pathdist.csv'

# Init variables and constants
#region
STATE_SIZE = 1
ACTION_SIZE = 9
MODEL_NAME = 'Model_Name'

# Exploration settings
epsilon = 1  # not a constant, going to be decayed
EPSILON_DECAY = 0.99975
MIN_EPSILON = 0.001
DISCOUNT = 0.99
MAX_STEP = 1000

# Environment settings
####################################################
EPISODES = 2000 
####################################################


####################################################
REPLAY_MEMORY_SIZE = 5000 # How many last steps to keep for model training
MIN_REPLAY_MEMORY_SIZE = 100
MINIBATCH_SIZE = 10  # How many steps (samples) to use for training
UPDATE_TARGET_EVERY = 5  # Terminal states (end of episodes)
####################################################

MEMORY_FRACTION = 0.20

#  Stats settings
AGGREGATE_STATS_EVERY = 50  # episodes
SHOW_PREVIEW = False
#endregion

# Init variables and constants

MIN_REWARD = -200  # For model save




## ENV

In [102]:
# Environment

def createLine(walls, SIZE_Y, SIZE_X):
    # Full Horizontal Line
    # for i in range(SIZE_X):
    #     walls[int(SIZE_Y/2),int(i)] = 1

    # Full Vertical Line
    # for i in range(SIZE_Y):
    # walls[int(i), int(SIZE_X/2)] = 1

    # Segment 1
    for i in range(SIZE_Y//10, SIZE_Y//3):
        walls[i, SIZE_X//2] = 1

    # Segment 2
    for i in range(SIZE_X//2, SIZE_X//2+SIZE_X//6):
        walls[SIZE_Y//10, i] = 1

    # Segment 3
    for i in range(SIZE_Y//10, SIZE_Y//4):
        walls[i, SIZE_X//2+SIZE_X//6] = 1

    # Segment 4
    for i in range(SIZE_X//2-SIZE_X//6, SIZE_X//2+1):
        walls[SIZE_Y//3, i] = 1

    # Segment 5
    for i in range(SIZE_Y//3, SIZE_Y//3+SIZE_Y//4):
        walls[i, SIZE_X//2-SIZE_X//6] = 1

    # Segment 6
    for i in range(SIZE_X//2-SIZE_X//6, SIZE_X//2+SIZE_X//6):
        walls[SIZE_Y//3+SIZE_Y//4, i] = 1

    # Segment 7
    for i in range(SIZE_Y//3+SIZE_Y//4, SIZE_Y//3+SIZE_Y//4+SIZE_Y//5):
        walls[i, SIZE_X//2+SIZE_X//6] = 1

    # Segment 8
    for i in range(SIZE_X//2+SIZE_X//6, SIZE_X//2+SIZE_X//6+SIZE_X//6):
        walls[SIZE_Y//3+SIZE_Y//4+SIZE_Y//5, i] = 1

    # Segment 9
    for i in range(SIZE_Y-SIZE_Y//3, SIZE_Y):
        walls[i, SIZE_X//2] = 1

    # Segment 10
    for i in range(SIZE_Y-SIZE_Y//4, SIZE_Y):
        walls[i, SIZE_X//2-SIZE_X//4] = 1

    # Segment 11
    for i in range(0, SIZE_X//6):
        walls[SIZE_Y//5, i] = 1

    # Segment 12
    for i in range(SIZE_Y//5, SIZE_Y//5+SIZE_Y//5):
        walls[i, SIZE_X//6] = 1

    # Segment 13
    for i in range(0, SIZE_X//6):
        walls[SIZE_Y//5+SIZE_Y//3, i] = 1

    # Segment 14
    for i in range(0, SIZE_Y//3):
        walls[i, SIZE_X-SIZE_X//6] = 1

    # Segment 15
    for i in range(SIZE_X-SIZE_X//10, SIZE_X):
        walls[SIZE_Y//3, i] = 1

    # Segment 16
    for i in range(SIZE_X-SIZE_X//9, SIZE_X):
        walls[SIZE_Y//2+SIZE_Y//10, i] = 1

    # Segment 17
    for i in range(SIZE_Y//2+SIZE_Y//10, SIZE_Y//2+SIZE_Y//10+SIZE_Y//4):
        walls[i, SIZE_X-SIZE_X//9] = 1

    # Segment 18
    for i in range(SIZE_Y//2+SIZE_Y//10+SIZE_Y//3, SIZE_Y):
        walls[i, SIZE_X-SIZE_X//9] = 1

    return walls


class EnvObject:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return f"{self.x}, {self.y}"

    def __sub__(self, other):
        return (self.x - other.x, self.y - other.y)

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y


class Drone(EnvObject):
    def __init__(self, x, y):
        super().__init__(x, y)

    def action(self, choice, walls = None):
        '''
        Gives us 9 total movement options. (0,1,2,3,4,5,6,7,8)
        '''
        if choice == 0:
            x = 0
            y = 0
            
        elif choice == 1:
            x=-1
            y=-1
        elif choice == 2:
            x=-1
            y=1
        elif choice == 3:
            x=1
            y=-1
        elif choice == 4:
            x=1
            y=1
            
        elif choice == 5:
            x=1
            y=0
        elif choice == 6:
            x=-1
            y=0
            
        elif choice == 7:
            x=0
            y=1
        elif choice == 8:
            x=0
            y=-1
        
        self.move(x=x, y=y, walls=walls)
            

    def move(self, x=0, y=0, walls = None):
        #if no value for x or y, stay
        
        x,y = self.collisionCheck(x,y,walls)
        self.x += x
        self.y += y

            
    def collisionCheck(self, x=0, y=0, walls=None):
        #checking for out of bounds
        predict_x = self.x + x
        predict_y = self.y + y
        if predict_x < 0 or predict_x > SIZE_X-1:
            x = 0
        if predict_y < 0 or predict_y > SIZE_Y-1:
            y = 0
            
        
        # Check for collision with walls    
        if walls is None:
            return x,y
        elif walls[self.y+y][self.x+x] == 1:
            return 0,0
        else:
            return x,y
        
        # # Several edge cases exist so we'll implement a simpler system
        # for i in range(np.sign(x), x + np.sign(x), np.sign(x)):
        #     if walls[self.y][self.x+i] == 1:
        #         x = i - np.sign(x)
        #         break
            
        # for j in range(np.sign(y), y + np.sign(y), np.sign(y)):
        #     if walls[self.y+j][self.x] == 1:
        #         y = i - np.sign(y)
        #         break
        

class Target(EnvObject):
    def __init__(self, x, y):
        super().__init__(x, y)


class DroneEnv:
    # Define Parameters
    SIZE_X = SIZE_X
    SIZE_Y = SIZE_Y
    ENV_COLOR = (20, 52, 89)
    WALLS_COLOR = (77, 77, 234)
    DRONE_COLOR = (234, 222, 53)
    TARGET_COLOR = (132, 234, 53)
    space = np.zeros((SIZE_Y, SIZE_X, 3), dtype=np.uint8)
    walls = createLine(
        np.zeros((SIZE_Y, SIZE_X), dtype=np.uint8), SIZE_Y, SIZE_X)
    pathDist = pd.read_csv(pathDist_path, header=None, dtype='Int32').values
    
    def reset(self):
        self.agent_1 = Drone(self.SIZE_X-self.SIZE_X//11, self.SIZE_Y//10)
        self.agent_2 = Drone(self.SIZE_X-self.SIZE_X//20,
                             self.SIZE_Y//2-self.SIZE_Y//20)
        self.agent_3 = Drone(self.SIZE_X-self.SIZE_X//13,
                             self.SIZE_Y-self.SIZE_Y//11)
        self.target = Target(self.SIZE_X//10, self.SIZE_Y//2-self.SIZE_Y//20)

        self.episode_step = 0

        observation = (self.pathDist[self.agent_1.y][self.agent_1.x])
        return observation

    def step(self, action, observation):
        reward = 0
        done= False
        self.episode_step += 1
        self.agent_1.action(action, self.walls)

        new_observation = (self.pathDist[self.agent_1.y][self.agent_1.x])

        if self.agent_1 == self.target:
            reward = 100
            done = True
        elif self.episode_step >= MAX_STEP:
            reward = -10
            done = True
        elif new_observation < observation:
            reward = observation - new_observation - 1
            done = False
        elif new_observation > observation:
            reward = observation - new_observation - 1
            done = False

        return new_observation, reward, done

    def is_wall(self, action):
        if self.walls[self.agent_1.y+action][self.agent_1.x+action] == 1:
            return True

    def visualize(self):
        for i in range(self.SIZE_Y):
            for j in range(self.SIZE_X):
                if self.walls[i][j] == 1:
                    self.space[i][j] = self.WALLS_COLOR
                else:
                    self.space[i][j] = self.ENV_COLOR

        self.space[self.agent_1.y][self.agent_1.x] = self.DRONE_COLOR
        self.space[self.target.y][self.target.x] = self.TARGET_COLOR

    def render(self):
        self.visualize()
        img = Image.fromarray(self.space, 'RGB')
        # img = img.resize((1200, 800), resample=Image.Resampling.BOX)
        cv2.imshow("image", np.array(img))  # show it!
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def saveImage(self, image_path, episode='e', step='s'):
        self.visualize()
        if not os.path.isdir(image_path):
            os.makedirs(image_path)

        img = Image.fromarray(self.space, 'RGB')
        # img = img.resize((1200, 800), resample=Image.Resampling.BOX)
        img_rgb = img.convert('RGB')
        img_rgb = img_rgb.save(f'{image_path}/episode_{episode}/image_{episode}_{step}.png')
        

    def test(self):
        print('test')


## DQNAgent

In [103]:
# DQN Agent
class ModifiedTensorBoard(TensorBoard):    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        self.writer = tf.summary.create_file_writer(self.log_dir)
        self._log_write_dir = self.log_dir

    def set_model(self, model):
        self.model = model

        self._train_dir = os.path.join(self._log_write_dir, 'train')
        self._train_step = self.model._train_counter

        self._val_dir = os.path.join(self._log_write_dir, 'validation')
        self._val_step = self.model._test_counter

        self._should_write_train_graph = False

    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    def on_batch_end(self, batch, logs=None):
        pass

    def on_train_end(self, _):
        pass

    def update_stats(self, **stats):
        with self.writer.as_default():
            for key, value in stats.items():
                tf.summary.scalar(key, value, step = self.step)
                self.writer.flush()

# Main Agent class
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size= state_size
        self.action_size = action_size

        # Main model
        self.model = self.create_model()

        # Target network
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())

        # An array with last n steps for training
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)

        # Custom tensorboard object
        self.tensorboard = ModifiedTensorBoard(
            log_dir="logs/{}-{}".format(MODEL_NAME, int(time.time())))

        # Used to count when to update target network with main network's weights
        self.target_update_counter = 0
        


    def create_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss="mse", optimizer=Adam(
            lr=0.001), metrics=['accuracy'])
        return model

    # Adds step's data to a memory replay array
    # (observation space, action, reward, new observation space, done)
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)

    # Trains main network every step during episode
    def train(self, terminal_state, step):

        # Start training only if certain number of samples is already saved
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return

        # Get a minibatch of random samples from memory replay table
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)

        # Get current states from minibatch, then query NN model for Q values
        current_states = np.array([transition[0]
                                  for transition in minibatch])
        current_qs_list = self.model.predict(current_states, verbose=0)

        # Get future states from minibatch, then query NN model for Q values
        # When using target network, query it, otherwise main network should be queried
        new_current_states = np.array(
            [transition[3] for transition in minibatch])
        future_qs_list = self.target_model.predict(new_current_states,verbose=0)

        X = []
        y = []

        # Now we need to enumerate our batches
        for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):

            # If not a terminal state, get new q from future states, otherwise set it to 0
            # almost like with Q Learning, but we use just part of equation here
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward

            # Update Q value for given state
            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            # And append to our training data
            X.append(current_state)
            y.append(current_qs)

        # Fit on all samples as one batch, log only on terminal state
        self.model.fit(np.array(X), np.array(y), batch_size=MINIBATCH_SIZE, verbose=0,
                       shuffle=False, callbacks=[self.tensorboard] if terminal_state else None)

        # Update target network counter every episode
        if terminal_state:
            self.target_update_counter += 1

        # If counter reaches set value, update target network with weights of main network
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights(),verbose=0)
            self.target_update_counter = 0

    # Queries main network for Q values given current observation space (environment state)
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, STATE_SIZE),verbose = 0)[0]


# Training

## Training Params

In [104]:
# Parameters for training


# Exploration settings
epsilon = 1  # not a constant, going to be decayed
EPSILON_DECAY = 0.995
MIN_EPSILON = 0.001
DISCOUNT = 0.99
MAX_STEP = 500

# Environment settings
####################################################
EPISODES = 2000 
####################################################



In [105]:
MODEL_NAME = f'-DQN_e{EPSILON_DECAY}_m{MAP_X}x{MAP_Y}'

## Training Code

In [106]:

# For stats
ep_rewards = [-200]

# For more repetitive results
random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

# Create models folder
date = time.strftime("%Y_%m_%d-%H_%M")
result_path = date + MODEL_NAME
image_path = os.path.join('images', result_path)
model_path = os.path.join('models', result_path)
episode_path = os.path.join(image_path, 'episode_')
if not os.path.isdir(model_path):
    os.makedirs(model_path)

if not os.path.isdir(image_path):
    os.makedirs(image_path)

os.makedirs(episode_path + str(1))
for episode in range(int(EPISODES/50)):
    os.makedirs(episode_path + str((episode+1)*50))

# MAIN PROGRAM

# A. Initialize DQNAgent (include policy network, target network, and replay memory capacity)
agent = DQNAgent(STATE_SIZE, ACTION_SIZE)
env = DroneEnv()


for episode in tqdm(range(1, EPISODES + 1), ascii=True, unit='episodes'): #tqdms is a progress bar
    
    # // Update tensorboard step every episode
    agent.tensorboard.step = episode

    # // Restarting episode - reset episode reward and step number
    episode_reward = 0
    step = 1
    
    # 1. Initialize the starting state
    current_state = env.reset()
    
    # 2. Reset flag and start iterating until episode ends
    done = False
    while not done:
        # a. Explore Exploit Tradeoff
        # b. Execute the action in the environment
        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(agent.get_qs(current_state))
        else:
            # Get random action
            action = np.random.randint(0, ACTION_SIZE)
            # action = np.argmax(agent.get_qs(current_state))
        
        # c. Observe reward and next state
        new_state, reward, done = env.step(action, current_state)
        episode_reward += reward
        
        # // Render
        # if SHOW_PREVIEW and not episode % AGGREGATE_STATS_EVERY:
        #     env.render()
            
        # d. Store experience in replay memory
        agent.update_replay_memory((current_state, action, reward, new_state, done))
        
        # e. Train the agent
        agent.train(done, step)
        current_state = new_state
        if episode % 50 == 0 or episode == 1:
            env.saveImage(image_path, episode, step)
        step += 1
    
    # 3. Tensorboard, append episode reward to a list and log stats (every given number of episodes)
    ep_rewards.append(episode_reward)
    if not episode % AGGREGATE_STATS_EVERY or episode == 1:
        average_reward = sum(ep_rewards[-AGGREGATE_STATS_EVERY:])/len(ep_rewards[-AGGREGATE_STATS_EVERY:])
        min_reward = min(ep_rewards[-AGGREGATE_STATS_EVERY:])
        max_reward = max(ep_rewards[-AGGREGATE_STATS_EVERY:])
        agent.tensorboard.update_stats(reward_avg=average_reward, reward_min=min_reward, reward_max=max_reward, epsilon=epsilon)

        # Save model, but only when min reward is greater or equal a set value
        if episode % 50 == 0 or episode == 1:
            agent.model.save(f'models/{MODEL_NAME}__{max_reward:_>7.2f}max_{average_reward:_>7.2f}avg_{min_reward:_>7.2f}min__{int(time.time())}.model')

    if epsilon > MIN_EPSILON:
        epsilon *= EPSILON_DECAY
        epsilon = max(MIN_EPSILON, epsilon)

    print('steps: ', step, 'epsilon: ', epsilon, 'episode_reward: ', episode_reward)
        

e:\Project\uav-rl-paper\.venv\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
  0%|          | 0/2000 [00:00<?, ?episodes/s]

1/1 - 1s - loss: 47.3283 - accuracy: 1.0000 - 909ms/epoch - 909ms/step
1/1 - 0s - loss: 52.1198 - accuracy: 0.6000 - 11ms/epoch - 11ms/step
1/1 - 0s - loss: 39.4827 - accuracy: 0.6000 - 9ms/epoch - 9ms/step
1/1 - 0s - loss: 13.4974 - accuracy: 0.6000 - 9ms/epoch - 9ms/step
1/1 - 0s - loss: 50.2572 - accuracy: 0.5000 - 9ms/epoch - 9ms/step
1/1 - 0s - loss: 46.5571 - accuracy: 0.3000 - 10ms/epoch - 10ms/step
1/1 - 0s - loss: 49.1543 - accuracy: 0.6000 - 9ms/epoch - 9ms/step
1/1 - 0s - loss: 49.8137 - accuracy: 0.4000 - 8ms/epoch - 8ms/step
1/1 - 0s - loss: 68.2525 - accuracy: 0.2000 - 10ms/epoch - 10ms/step
1/1 - 0s - loss: 64.3059 - accuracy: 0.1000 - 11ms/epoch - 11ms/step
1/1 - 0s - loss: 45.2529 - accuracy: 0.6000 - 7ms/epoch - 7ms/step
1/1 - 0s - loss: 55.9147 - accuracy: 0.3000 - 9ms/epoch - 9ms/step
1/1 - 0s - loss: 42.3958 - accuracy: 0.3000 - 9ms/epoch - 9ms/step
1/1 - 0s - loss: 39.3476 - accuracy: 0.1000 - 10ms/epoch - 10ms/step
1/1 - 0s - loss: 46.0203 - accuracy: 0.2000 - 8m

  0%|          | 0/2000 [00:36<?, ?episodes/s]


KeyboardInterrupt: 

array([-0.09048758,  7.632888  , -1.901063  ,  0.8824748 , -2.6757767 ,
       12.894743  ,  1.0560378 , 15.565678  ,  4.344231  ], dtype=float32)